# Summary

* Plot the fits
* Plot the uncertainty on the signal yield

In [1]:
import sys
sys.path.append('../../FourVector')
sys.path.append('../project')

from FourVector import FourVector
from ThreeVector import ThreeVector

from FutureColliderTools import SmearVertex, GetCorrectedMass, GetMissingMass2, GetQ2
from FutureColliderDataLoader import LoadData_KMuNu, LoadData_DsMuNu
from FutureColliderVariables import *
from FutureColliderFitter import FutureFitter

import numpy as np
import ROOT

ROOT.enableJSVis()
ROOT.gStyle.SetOptStat(0)

Welcome to JupyROOT 6.08/02


# Make the canvases for each plot

In [2]:
File_Fits = ROOT.TFile.Open("../output/FitterOutput.root", "READ")

Canv_Fits = {}
Canv_Fits["Ds"] = ROOT.TCanvas("c_Ds", "", 900, 900)
Canv_Fits["K"]  = ROOT.TCanvas("c_K", "", 900, 900)
Canv_Fits["Ds"].Divide(3,3)
Canv_Fits["K"] .Divide(3,3)

Legends = {}
Texts = []

for Mode in ["K", "Ds"]:
    it = 1
    for resolution in np.linspace(0.3, 1.0, 8):
        canvas = File_Fits.Get("c_Fit_{mode}_{resolution}".format(mode = Mode, resolution = resolution))
        Canv_Fits[Mode].cd(it)
        for prim in canvas.GetListOfPrimitives():
            if prim.GetName() == "TPave":
                Legends[Mode] = prim.Clone()
                canvas.GetListOfPrimitives().Remove(prim)
                break
        Text = ROOT.TPaveText(0.02, 0.8, 0.3, 0.98, "NDC")
        Text.AddText("{0} #sigma LHCb".format(resolution))
        Text.SetBorderSize(0)
        Text.SetFillColor(0)
        canvas.DrawClonePad()
        
        Text.Draw()
        Texts += [Text]
        it+=1
        
    Canv_Fits[Mode].cd(9)
    Legends[Mode].SetX1(0)
    Legends[Mode].SetX2(1)
    Legends[Mode].SetY1(0)
    Legends[Mode].SetY2(1)
    Legends[Mode].Draw()


In [3]:
Canv_Fits["Ds"].Draw()

In [4]:
Canv_Fits["K"].Draw()

# Print the results of the fit

In [5]:
import pickle
Results = pickle.load( open("../output/FitResults.p") )

print "Results for K Mu Nu Fit:"
print Results["K"]
print "\nResults for Ds Mu Nu Fit:"
print Results["Ds"]

Results for K Mu Nu Fit:
+---------------------+-----------------+---------------+
| Vertex Error / LHCb | Signal Fraction |   Error [%]   |
+---------------------+-----------------+---------------+
|         0.0         |  0.159658979541 | 1.63507107963 |
|         0.1         |  0.159132771093 |  1.9299029019 |
|         0.2         |  0.159972142134 | 2.16895443932 |
|         0.3         |  0.16000528803  | 2.12937142758 |
|         0.4         |  0.154545246657 | 243.791889847 |
|         0.5         |  0.158118773897 | 2.74401476856 |
|         0.6         |  0.158142900098 | 3.01320873939 |
|         0.7         |  0.155373214193 | 3.30174816081 |
|         0.8         |  0.16220540394  | 3.67975354972 |
|         0.9         |  0.158740520893 | 3.79864908641 |
|         1.0         |  0.158143449301 | 3.96826862127 |
+---------------------+-----------------+---------------+

Results for Ds Mu Nu Fit:
+---------------------+-----------------+---------------+
| Vertex Error / LHC

In [6]:
Graph_K  = File_Fits.Get("Error_Budget_K")
Graph_Ds = File_Fits.Get("Error_Budget_Ds")

Graph_K.SetLineStyle(2)
Graph_K.SetMarkerStyle(4)

Graph_Ds.SetLineStyle(2)
Graph_Ds.SetMarkerStyle(4)


# Plot the error budget for $B_s \to k^- \mu^+ \nu_{\mu}$

In [7]:


canv_Budget = ROOT.TCanvas("c_graph", "", 900, 900)
Graph_K.Draw()
canv_Budget.Draw()

# Plot the error budget for $B_s \to D_s^- \mu^+ \nu_{\mu}$

In [8]:
canv_Budget.Clear()
Graph_Ds.Draw()
canv_Budget.Draw()

## There are some fits which terminate early and produce some eroneous errors. 
## This needs to be investigated